In [6]:
# locating files 
import os

os.getcwd()

'$LAB/ssackey/notebooks'

In [7]:
import pandas as pd

gff_file = "$LAB/ssackey/data/example_ncbi/genomic.gff"

In [83]:
# Load GFF
gff_cols = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
gff = pd.read_csv(gff_file, sep="\t", comment='#', names=gff_cols) 

# Filter for protein-coding regions (CDS)
cds = gff[gff['type'] == 'CDS'].copy()

In [82]:
print(gff)

      NZ_OZ061239.1            RefSeq      region     1  2353226  .  + ..1  \
0     NZ_OZ061239.1            RefSeq  pseudogene    31     1342  .  +   .   
1     NZ_OZ061239.1  Protein Homology         CDS    31     1342  .  +   0   
2     NZ_OZ061239.1            RefSeq        gene  1532     2671  .  +   .   
3     NZ_OZ061239.1  Protein Homology         CDS  1532     2671  .  +   0   
4     NZ_OZ061239.1            RefSeq        gene  2976     3206  .  +   .   
...             ...               ...         ...   ...      ... .. ..  ..   
5139  NZ_OZ061244.1  Protein Homology         CDS  6845     7189  .  -   0   
5140  NZ_OZ061244.1            RefSeq        gene  7183     7446  .  -   .   
5141  NZ_OZ061244.1  Protein Homology         CDS  7183     7446  .  -   0   
5142  NZ_OZ061244.1            RefSeq  pseudogene  7532     7785  .  +   .   
5143  NZ_OZ061244.1  Protein Homology         CDS  7532     7785  .  +   0   

     ID=NZ_OZ061239.1:1..2353226;Dbxref=taxon:33962;Name=1;chro

In [20]:
from glob import glob

In [30]:
virus_path = glob("$LAB/ssackey/data/example_genomad/*find_proviruses/*provirus.tsv")[0] 
# this is the virus genomad file which has information about the start and end location of the virus sequence 

In [33]:
df_virus = pd.read_csv(virus_path, sep = "\t")
df_virus.head()

,seq_name,source_seq,start,end,length,n_genes,v_vs_c_score,in_seq_edge,integrases
0,NZ_OZ061239.1|provirus_1433333_1479268,NZ_OZ061239.1,1433333,1479268,45936,56,55.9504,False,NZ_OZ061239.1|provirus_1433333_1479268_1464


In [38]:
provirus_file = "$LAB/ssackey/data/example_genomad/genomad_GCF_964062605.1_Lentilactobacillus_kefiri_CIRM-BIA2556_genomic/GCF_964062605.1_Lentilactobacillus_kefiri_CIRM-BIA2556_genomic_find_proviruses/GCF_964062605.1_Lentilactobacillus_kefiri_CIRM-BIA2556_genomic_provirus.tsv"

In [89]:
def parse_attributes(attr_str, key):
    for entry in attr_str.split(';'):
        if entry.startswith(key + "="):
            return entry.split('=', 1)[1]
    return None

gff['protein_id'] = gff['attributes'].apply(lambda x: parse_attributes(x, 'protein_id') or parse_attributes(x, 'ID'))
gff['product'] = gff['attributes'].apply(lambda x: parse_attributes(x, 'product'))

In [90]:
results = []

for _, row in df_virus.iterrows():
    region_start = int(row['start'])
    region_end = int(row['end'])
    contig = row['source_seq']
    
    matching = cds[
        (cds['seqid'] == contig) &
        (cds['start'] >= region_start) &
        (cds['end'] <= region_end)
    ].copy()
    
    matching['region_type'] = 'Provirus'
    matching['region_start'] = region_start
    matching['region_end'] = region_end
    
    results.append(matching)

In [91]:
print(results)

[              seqid            source type    start      end score strand  \
2933  NZ_OZ061239.1  Protein Homology  CDS  1433333  1433584     .      -   
2935  NZ_OZ061239.1  Protein Homology  CDS  1433584  1434408     .      -   
2937  NZ_OZ061239.1      GeneMarkS-2+  CDS  1434576  1434887     .      -   
2939  NZ_OZ061239.1  Protein Homology  CDS  1435264  1435437     .      -   
2941  NZ_OZ061239.1  Protein Homology  CDS  1435455  1435865     .      -   
2943  NZ_OZ061239.1  Protein Homology  CDS  1435880  1437703     .      -   
2945  NZ_OZ061239.1  Protein Homology  CDS  1437729  1440065     .      -   
2947  NZ_OZ061239.1  Protein Homology  CDS  1440114  1440863     .      -   
2949  NZ_OZ061239.1  Protein Homology  CDS  1440869  1442119     .      -   
2951  NZ_OZ061239.1  Protein Homology  CDS  1442112  1447688     .      -   
2953  NZ_OZ061239.1  Protein Homology  CDS  1447898  1448275     .      -   
2955  NZ_OZ061239.1  Protein Homology  CDS  1448334  1448957     .      -  

In [92]:
# using geNomad to pull information from NCBI files (GFF)
matching_features = []

for _, row in df_virus.iterrows():
    region_contig = row['source_seq']
    region_start = int(row['start'])
    region_end = int(row['end'])

    matching = gff[
        (gff['seqid'] == region_contig) &
        (gff['start'] >= region_start) &
        (gff['end'] <= region_end)
    ].copy()

    matching['region_type'] = 'Provirus'
    matching['region_start'] = region_start
    matching['region_end'] = region_end

    matching_features.append(matching)



In [93]:
# display resulting dataframe 
virus_sequence_features_df = pd.concat(matching_features)
virus_sequence_features_df = virus_sequence_features_df[['region_type', 'seqid', 'region_start', 'region_end', 'type', 'start', 'end', 'strand', 'attributes']]


In [94]:
print(virus_sequence_features_df)

     region_type          seqid  region_start  region_end        type  \
2932    Provirus  NZ_OZ061239.1       1433333     1479268        gene   
2933    Provirus  NZ_OZ061239.1       1433333     1479268         CDS   
2934    Provirus  NZ_OZ061239.1       1433333     1479268  pseudogene   
2935    Provirus  NZ_OZ061239.1       1433333     1479268         CDS   
2936    Provirus  NZ_OZ061239.1       1433333     1479268        gene   
...          ...            ...           ...         ...         ...   
3043    Provirus  NZ_OZ061239.1       1433333     1479268        tRNA   
3044    Provirus  NZ_OZ061239.1       1433333     1479268        exon   
3045    Provirus  NZ_OZ061239.1       1433333     1479268        gene   
3046    Provirus  NZ_OZ061239.1       1433333     1479268        tRNA   
3047    Provirus  NZ_OZ061239.1       1433333     1479268        exon   

        start      end strand  \
2932  1433333  1433584      -   
2933  1433333  1433584      -   
2934  1433584  1434408  

In [95]:
# extract product from attributes
def parse_attributes(attr_str, key):
    for entry in attr_str.split(';'):
        if entry.startswith(key + "="):
            return entry.split('=', 1)[1]
    return None

In [96]:
gff['product'] = gff['attributes'].apply(lambda x: parse_attributes(x, 'product'))

In [ ]:
display(